In [2]:
import pandas as pd
import numpy as np
import ETL_functions as etf
import ast


In [3]:
df_sg = pd.read_json(r"../Datasets/steam_games.json/output_steam_games.json" ,lines=True) #archivo steam_games

    transformo las siguientes columnas a tipo str ya que es dificil manejar listas y tira errores en bastantes metodos de pandas

In [4]:
df_sg["genres"] = df_sg["genres"].astype(str)
df_sg["tags"] = df_sg["tags"].astype(str)
df_sg["specs"] = df_sg["specs"].astype(str)

In [5]:
etf.data_review(df_sg)


Total rows:  120445

Total full null rows:  0

Total duplicated rows: 88309


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,publisher,"[<class 'NoneType'>, <class 'str'>]",20.00,24083,80.00,96362
1,genres,[<class 'str'>],100.00,120445,0.00,0
2,app_name,"[<class 'NoneType'>, <class 'str'>]",26.68,32133,73.32,88312
3,title,"[<class 'NoneType'>, <class 'str'>]",24.98,30085,75.02,90360
4,url,"[<class 'NoneType'>, <class 'str'>]",26.68,32135,73.32,88310
5,release_date,"[<class 'NoneType'>, <class 'str'>]",24.96,30068,75.04,90377
6,tags,[<class 'str'>],100.00,120445,0.00,0
7,reviews_url,"[<class 'NoneType'>, <class 'str'>]",26.68,32133,73.32,88312
8,specs,[<class 'str'>],100.00,120445,0.00,0
9,price,"[<class 'NoneType'>, <class 'float'>, <class '...",25.54,30758,74.46,89687


    para esta funcion solo utilizaremos(en este df):
* id(para relacionar con la review)
* developer(va a ser por lo que vamos a filtrar/el dato que se pasa a la funcion)

se podria usar pero no la usare:
* publisher(en caso de no tener developer podria devolver esto como publisher y no como developer) #pero la funcion pide developer asi que no la usare

In [6]:
del df_sg["genres"]
del df_sg["app_name"]
del df_sg["title"]
del df_sg["url"]
del df_sg["release_date"]
del df_sg["tags"]
del df_sg["reviews_url"]
del df_sg["specs"]
del df_sg["price"]
del df_sg["early_access"]
del df_sg["publisher"]

    borramos todas estas columnas que no vamos a utilizar

## este es nuestro dataframe ahora
contiene las columnas que utilizare ,
pero sigue teniendo filas que no nos sirven

In [20]:
df_sg

,id,developer
88310,761140.0,Kotoshiro
88311,643980.0,Secret Level SRL
88312,670290.0,Poolians.com
88313,767400.0,彼岸领域
88314,773570.0,None
...,...,...
120440,773640.0,"Nikita ""Ghost_RUS"""
120441,733530.0,Sacada
120442,610660.0,Laush Dmitriy Sergeevich
120443,658870.0,"xropi,stev3ns"


In [21]:
etf.data_review(df_sg)


Total rows:  32133

Total full null rows:  0

Total duplicated rows: 0


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,id,[<class 'float'>],100.00,32132,0.00,1
1,developer,"[<class 'str'>, <class 'NoneType'>]",89.74,28835,10.26,3298


    con esto podemos ver la cantidad de null y filas duplicadas que en este caso son las mismas cantidades

## eliminacion de duplicados

    cualquier registro duplicado no resulta importante debido a que los datos ya se encuentran en el dataset 
    (en especial en este caso que tenemos muchas columnas que tienen todas la columnas en null)

In [9]:
df_sg.drop_duplicates(inplace=True)#elimino todos los duplicados dejando solo registros unicos y con ellos una columna llena de valores null que removere en la siguiente instruccion

    al dropear duplicados nos dejo con cada registro unico incluyendo  una de las lineas con todos sus valores nulos 

In [22]:
df_sg.head(3)

,id,developer
88310,761140.0,Kotoshiro
88311,643980.0,Secret Level SRL
88312,670290.0,Poolians.com


    como en este caso quedo en la primera linea la dropeare indicando el indice

In [12]:
df_sg.drop(0,inplace=True)#borramos la unica columna que no poseia datos que quedaba

In [23]:
df_sg.head()

,id,developer
88310,761140.0,Kotoshiro
88311,643980.0,Secret Level SRL
88312,670290.0,Poolians.com
88313,767400.0,彼岸领域
88314,773570.0,None


In [24]:
etf.data_review(df_sg)


Total rows:  32133

Total full null rows:  0

Total duplicated rows: 0


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,id,[<class 'float'>],100.00,32132,0.00,1
1,developer,"[<class 'str'>, <class 'NoneType'>]",89.74,28835,10.26,3298


    ahora ya no tenemos filas completas con nulos o filas duplicadas lo que no quiere decir que no haya nulos
    como podemos comprobar en la ultima columna de el retorno de etf.data_review donde nos dice
|columna|cantidad de nulos|
|-------|-----------------|
|id| 1
|developer| 3298

## eliminacion de nulos

    borrare cualquier nulo en developer

In [25]:
mask = df_sg["developer"].isna() #

In [27]:
df_sg = df_sg[~mask] # uso ~ para invertir la condicion de el  filtro lo que traera todos los que developer no es nulo

    ahora tratare el nulo de id

In [29]:
df_sg.isna().sum()

id           1
developer    0
dtype: int64

In [30]:
df_sg.dropna(inplace=True)

C:\Users\delba\AppData\Local\Temp\ipykernel_7124\2810401005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sg.dropna(inplace=True)


In [32]:
etf.data_review(df_sg)


Total rows:  28834

Total full null rows:  0

Total duplicated rows: 0


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,id,[<class 'float'>],100.0,28834,0.0,0
1,developer,[<class 'str'>],100.0,28834,0.0,0


    paso id a int paraa perder la parte decimal

In [41]:
df_sg["id"] = df_sg["id"].astype(int)

C:\Users\delba\AppData\Local\Temp\ipykernel_7124\2301839919.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sg["id"] = df_sg["id"].astype(int)


In [58]:
df_sg["id"] = df_sg["id"].astype(str)

,id,developer
88310,761140,Kotoshiro
88311,643980,Secret Level SRL
88312,670290,Poolians.com
88313,767400,彼岸领域
88315,772540,Trickjump Games Ltd
...,...,...
120439,745400,Bidoniera Games
120440,773640,"Nikita ""Ghost_RUS"""
120441,733530,Sacada
120442,610660,Laush Dmitriy Sergeevich


    paso id a str para realizar  el merge

In [57]:
df_sg["id"] = df_sg["id"].astype(str)

C:\Users\delba\AppData\Local\Temp\ipykernel_7124\2447768868.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_sg["id"] = df_sg["id"].astype(str)


<hr>

In [35]:
df_rd = pd.read_parquet("../app/reviews_dict.parquet") #este archivo contiene las reviews y analisis de sentimiento

In [36]:
df_rd

,posted,item_id,recommend,review,sentimiento
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,2
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,2
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,2
0,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,2
1,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,0
...,...,...,...,...,...
2,Posted July 10.,70,True,a must have classic from steam definitely wort...,2
3,Posted July 8.,362890,True,this game is a perfect remake of the original ...,2
0,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,2
1,Posted July 20.,730,True,:D,2


    en este caso solo usarenos dos columnas:
* item_id(para relacionarlo con id)
* sentimiento(para sacar las cantidades)

In [38]:
del df_rd["posted"]
del df_rd["recommend"]
del df_rd["review"]

    pasamos item_id a str para poder hacer el merge

In [50]:
df_rd["item_id"] = df_rd["item_id"].astype(str)

In [39]:
df_rd

,item_id,sentimiento
0,1250,2
1,22200,2
2,43110,2
0,251610,2
1,227300,0
...,...,...
2,70,2
3,362890,2
0,273110,2
1,730,2


    con esto conservamos las columnas que necesitamos

In [44]:
etf.data_review(df_rd)


Total rows:  58424

Total full null rows:  0

Total duplicated rows: 51959


,Column,dType,No_Null_%,No_Null_Qty,Null_%,Null_Qty
0,item_id,[<class 'str'>],100.0,58424,0.0,0
1,sentimiento,[<class 'int'>],100.0,58424,0.0,0


    podemos notar  que esta perfectamente limpio esto debido a que el archivo review_dict es un archivo ya procesado que obtuvimos de user_reviews

<hr>

In [52]:
merge = df_rd.merge(df_sg,how="inner",left_on="item_id",right_on="id")

In [53]:
merge

,item_id,sentimiento,id,developer
0,1250,2,1250,Tripwire Interactive
1,1250,2,1250,Tripwire Interactive
2,1250,2,1250,Tripwire Interactive
3,1250,1,1250,Tripwire Interactive
4,1250,1,1250,Tripwire Interactive
...,...,...,...,...
49614,307130,2,307130,Legend Studio
49615,209120,0,209120,"Capcom U.S.A., Inc."
49616,220090,0,220090,SkyGoblin
49617,262850,0,262850,SkyGoblin


In [54]:
merge.to_parquet(r"../app/df_sentimientos.parquet")

<hr>

## funcion

In [59]:
def contar_sentimientos_por_desarrollador(developer):
    # Filtra el DataFrame para obtener solo las filas correspondientes al desarrollador dado
    #carga el df sentiment analisis

    df_temporal = merge[merge['developer'] == developer]#filtra el df donde las columnas correspondan al input

    # Cuenta la cantidad de sentimientos 0, 1 y 2
    count_sentimiento_0 = df_temporal[df_temporal['sentimiento'] == 0].count()
    count_sentimiento_1 = df_temporal[df_temporal['sentimiento'] == 1].count()
    count_sentimiento_2 = df_temporal[df_temporal['sentimiento'] == 2].count()

    # Devuelve los resultados
    return {
        'sentimiento_0': count_sentimiento_0.values[0],
        'sentimiento_1': count_sentimiento_1.values[0],
        'sentimiento_2': count_sentimiento_2.values[0]
    }

# Ejemplo de uso
developer_stats = contar_sentimientos_por_desarrollador('Tripwire Interactive')
print(developer_stats)

{'sentimiento_0': 115, 'sentimiento_1': 148, 'sentimiento_2': 303}


In [60]:
type(developer_stats)

dict

In [ ]:
@app.get("/sentiment_analysis")
async def sentiment_analysis( empresa_desarrolladora : str ): #Según la empresa desarrolladora, se devuelve un diccionario con el nombre de la desarrolladora como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor.
    # Filtra el DataFrame para obtener solo las filas correspondientes al desarrollador dado
    #carga el df sentiment analisis

    df_temporal = df_sentimientos[df_sentimientos['developer'] == empresa_desarrolladora]#filtra el df donde las columnas correspondan al input

    # Cuenta la cantidad de sentimientos 0, 1 y 2
    count_sentimiento_0 = df_temporal[df_temporal['sentimiento'] == 0].count()
    count_sentimiento_1 = df_temporal[df_temporal['sentimiento'] == 1].count()
    count_sentimiento_2 = df_temporal[df_temporal['sentimiento'] == 2].count()


    # Devuelve los resultados
    return {'sentimiento_0': count_sentimiento_0.values[0],'sentimiento_1': count_sentimiento_1.values[0],'sentimiento_2': count_sentimiento_2.values[0]}